In [18]:
# Generate Products Data từ API

# Tạo dữ liệu products từ API và insert vào database

In [19]:
%pip install requests pandas python-dotenv beautifulsoup4 psycopg2-binary deep-translator

Note: you may need to restart the kernel to use updated packages.


In [20]:
import os
import json
import random
import requests
import psycopg2
from datetime import datetime, timedelta
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator

load_dotenv()

SUPABASE_CONNECTION_STRING = os.getenv("SUPABASE_CONNECTION_STRING")
API_PRODUCT_URL = "https://api.artic.edu/api/v1/artworks"

# Cấu hình: lấy 100-120 products
TARGET_PRODUCTS = 120  # Số lượng products mong muốn
LIMIT_PER_PAGE = 100   # API limit mỗi page

print("✅ Đã import tất cả thư viện")

✅ Đã import tất cả thư viện


## 1. Helper Functions

In [27]:
def translate_text(text, source='en', target='vi'):
    """Dịch text từ tiếng Anh sang tiếng Việt"""
    if not text or text.strip() == '':
        return text
    try:
        translator = GoogleTranslator(source=source, target=target)
        return translator.translate(text)
    except Exception as e:
        print(f"⚠️ Lỗi dịch text: {e}")
        return text

def translate_html_content(html_content, source='en', target='vi'):
    """Dịch nội dung HTML, giữ nguyên tag"""
    if not html_content:
        return html_content
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        translator = GoogleTranslator(source=source, target=target)
        
        for text_node in soup.find_all(string=True):
            if text_node.parent.name in ['script', 'style']:
                continue
            
            text_stripped = text_node.strip()
            if text_stripped:
                translated = translator.translate(text_stripped)
                text_node.replace_with(translated)
        
        return str(soup)
    except Exception as e:
        print(f"⚠️ Lỗi dịch HTML: {e}")
        return html_content

def generate_end_time(start_time_str):
    """Generate end_time sau start_time từ 5 ngày đến 1 tháng"""
    try:
        start_time = datetime.fromisoformat(start_time_str.replace('Z', '+00:00'))
        days_options = [5, 7, 14, 21, 30]
        days_to_add = random.choice(days_options)
        end_time = start_time + timedelta(days=days_to_add)
        return end_time.isoformat()
    except Exception as e:
        print(f"⚠️ Lỗi generate end_time: {e}")
        return start_time_str

def generate_random_images(base_image_id, count=3):
    """Generate mảng images với các kích thước khác nhau từ IIIF API"""
    if not base_image_id:
        return ['https://via.placeholder.com/800'] * count
    
    # IIIF Image API - các sizes khác nhau
    sizes = [
        'full/843,/0/default.jpg',   # Large
        'full/400,/0/default.jpg',   # Medium
        'full/200,/0/default.jpg',   # Small/Thumbnail
    ]
    
    images = []
    for size in sizes[:count]:
        image_url = f"https://www.artic.edu/iiif/2/{base_image_id}/{size}"
        images.append(image_url)
    
    # Nếu cần thêm images, thêm các rotation variants
    while len(images) < count:
        angle = random.choice([90, 180, 270])
        image_url = f"https://www.artic.edu/iiif/2/{base_image_id}/full/400,/{angle}/default.jpg"
        images.append(image_url)
    
    return images

def generate_prices():
    """Generate các giá theo mệnh giá Việt Nam"""
    start_price = random.randint(50, 5000) * 10000
    step_price = random.randint(10, 200) * 10000
    buy_now_price = start_price + random.randint(500, 2000) * 10000
    
    return {
        'start_price': start_price,
        'current_price': start_price,
        'step_price': step_price,
        'buy_now_price': buy_now_price
    }

def get_seller_ids(cursor):
    """Lấy seller_ids từ bảng users, ưu tiên user_id 18 và 19"""
    cursor.execute("SELECT user_id FROM users WHERE user_id IN (18, 19) ORDER BY user_id")
    primary_sellers = [row[0] for row in cursor.fetchall()]
    
    cursor.execute("SELECT user_id FROM users WHERE user_id NOT IN (18, 19) LIMIT 5")
    backup_sellers = [row[0] for row in cursor.fetchall()]
    
    return primary_sellers + backup_sellers

def get_cat2_ids(cursor):
    """Lấy cat2_ids từ bảng categories với điều kiện parent_id NOT NULL"""
    cursor.execute("SELECT id FROM categories WHERE parent_id IS NOT NULL")
    return [row[0] for row in cursor.fetchall()]

def distribute_seller_ids(total_products, seller_ids):
    """Phân bổ seller_ids: user 18 và 19 mỗi người ~10 products, còn lại random"""
    result = []
    
    # User 18: 10 products đầu
    result.extend([18] * min(10, total_products))
    
    # User 19: 10 products tiếp
    if total_products > 10:
        result.extend([19] * min(10, total_products - 10))
    
    # Còn lại random từ all sellers
    remaining = total_products - len(result)
    if remaining > 0:
        result.extend([random.choice(seller_ids) for _ in range(remaining)])
    
    return result

print("✅ Các hàm hỗ trợ đã được định nghĩa!")

✅ Các hàm hỗ trợ đã được định nghĩa!


## 2. Kết nối Database

In [30]:
# Kết nối Supabase
if SUPABASE_CONNECTION_STRING:
    try:
        conn = psycopg2.connect(SUPABASE_CONNECTION_STRING)
        cursor = conn.cursor()
        
        cursor.execute("SELECT version();")
        db_version = cursor.fetchone()
        
        print("✅ Kết nối Supabase thành công!")
        print(f"📊 Database version: {db_version[0][:50]}...")
        
    except Exception as e:
        print(f"❌ Lỗi kết nối Supabase: {e}")
        conn = None
else:
    print("⚠️ Không tìm thấy SUPABASE_CONNECTION_STRING trong .env")

✅ Kết nối Supabase thành công!
📊 Database version: PostgreSQL 17.6 on aarch64-unknown-linux-gnu, comp...


In [31]:
# Fetch 100-120 products từ API (có thể cần gọi nhiều pages)
all_products = []
page = 3
TARGET_PRODUCTS = 30


print(f"🚀 Bắt đầu fetch {TARGET_PRODUCTS} products từ API...\n")

try:
    while len(all_products) < TARGET_PRODUCTS:
        print(f"📥 Đang fetch page {page}...")
        
        params = {
            'page': page,
            'limit': LIMIT_PER_PAGE
        }
        
        response = requests.get(API_PRODUCT_URL, params=params)
        response.raise_for_status()
        
        data = response.json()
        page_products = data.get('data', [])
        
        if not page_products:
            print("⚠️ Không còn dữ liệu từ API")
            break
        
        all_products.extend(page_products)
        print(f"   ✅ Đã fetch {len(page_products)} products (Total: {len(all_products)})")
        
        # Kiểm tra xem đã đủ chưa
        if len(all_products) >= TARGET_PRODUCTS:
            all_products = all_products[:TARGET_PRODUCTS]  # Trim về đúng số lượng
            break
        
        page += 1
    
    # Lưu vào file product.api.json
    api_result = {
        'data': all_products,
        'pagination': {
            'total': len(all_products),
            'total_pages': page,
            'current_page': page
        }
    }
    
    with open('product_addition.api.json', 'w', encoding='utf-8') as f:
        json.dump(api_result, f, ensure_ascii=False, indent=2)
    
    print(f"\n✅ Đã lưu {len(all_products)} sản phẩm vào product.api.json")
    print(f"📊 Total products: {len(all_products)}")
    print(f"📄 Total pages fetched: {page}")
    
except requests.exceptions.RequestException as e:
    print(f"❌ Lỗi khi gọi API: {e}")
except Exception as e:
    print(f"❌ Lỗi: {e}")

🚀 Bắt đầu fetch 30 products từ API...

📥 Đang fetch page 3...
   ✅ Đã fetch 100 products (Total: 100)

✅ Đã lưu 30 sản phẩm vào product.api.json
📊 Total products: 30
📄 Total pages fetched: 3


## 3. Fetch 100-120 Products từ API

In [32]:
# Generate SQL INSERT statements với seller_id và cat2_id từ database
try:
    with open('product_addition.api.json', 'r', encoding='utf-8') as f:
        api_data = json.load(f)
    
    products_list = api_data.get('data', [])
    print(f"📦 Đã load {len(products_list)} sản phẩm từ product.api.json\n")
    
    # Lấy seller_ids và cat2_ids từ database
    if conn:
        seller_ids = get_seller_ids(cursor)
        cat2_ids = get_cat2_ids(cursor)
        
        print(f"👥 Seller IDs: {seller_ids}")
        print(f"📁 Category IDs (parent_id NOT NULL): {cat2_ids}\n")
        
        if not seller_ids or not cat2_ids:
            print("❌ Không tìm thấy seller_ids hoặc cat2_ids trong database!")
            raise Exception("Missing sellers or categories")
        
        # Phân bổ seller_ids: user 18 và 19 mỗi người ~10 products
        distributed_sellers = distribute_seller_ids(len(products_list), seller_ids)
    else:
        print("❌ Không có kết nối database!")
        raise Exception("No database connection")
    
    # Generate SQL INSERT statements
    sql_statements = []
    sql_statements.append("-- Auto-generated INSERT statements for products table")
    sql_statements.append("-- Generated at: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    sql_statements.append("")
    
    for idx, product in enumerate(products_list, 1):
        print(f"🔄 Processing product {idx}/{len(products_list)}...")
        
        # Dịch product_name
        product_name = translate_text(product.get('title', 'Untitled'))
        
        # Lấy start_time từ updated_at
        start_time = product.get('updated_at', datetime.now().isoformat())
        
        # Generate end_time
        end_time = generate_end_time(start_time)
        
        # Dịch description
        description = translate_html_content(product.get('description', ''))
        
        # Generate product_images với sizes khác nhau
        image_id = product.get('image_id', '')
        product_images = generate_random_images(image_id, 3)
        
        # Random auto_extended
        auto_extended = random.choice([True, False])
        
        # Generate prices
        prices = generate_prices()
        
        # Lấy seller_id và cat2_id
        seller_id = distributed_sellers[idx - 1]
        cat2_id = random.choice(cat2_ids)
        
        # Escape single quotes
        product_name_escaped = product_name.replace("'", "''")
        description_escaped = description.replace("'", "''") if description else ''
        
        # Generate INSERT statement
        insert_sql = f"""INSERT INTO products (
    product_name, 
    seller_id, 
    cat2_id, 
    start_time, 
    end_time, 
    description, 
    product_images, 
    auto_extended,
    start_price,
    current_price,
    step_price,
    buy_now_price,
    bid_turns
) VALUES (
    '{product_name_escaped}',
    {seller_id},
    {cat2_id},
    '{start_time}',
    '{end_time}',
    '{description_escaped}',
    ARRAY{product_images}::TEXT[],
    {str(auto_extended).upper()},
    {prices['start_price']},
    {prices['current_price']},
    {prices['step_price']},
    {prices['buy_now_price']},
    0
);"""
        sql_statements.append(insert_sql)
    
    # Ghi vào file product.insert.sql
    with open('product_addition.insert.sql', 'w', encoding='utf-8') as f:
        f.write('\n\n'.join(sql_statements))
    
    print(f"\n✅ Đã tạo file product.insert.sql với {len(products_list)} câu lệnh INSERT")
    
    # Thống kê phân bổ sellers
    from collections import Counter
    seller_counts = Counter(distributed_sellers)
    print(f"\n📊 Phân bổ products theo seller:")
    for seller_id, count in sorted(seller_counts.items()):
        print(f"   Seller {seller_id}: {count} products")
    
except Exception as e:
    print(f"❌ Lỗi: {e}")
    import traceback
    traceback.print_exc()

📦 Đã load 30 sản phẩm từ product.api.json

👥 Seller IDs: [18, 19, 41, 42, 43, 44, 45]
📁 Category IDs (parent_id NOT NULL): [5, 6, 4, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 43, 44, 45, 46, 48, 51, 52, 53, 54, 55, 56, 57, 49, 50, 47]

🔄 Processing product 1/30...
🔄 Processing product 2/30...
🔄 Processing product 3/30...
🔄 Processing product 4/30...
🔄 Processing product 5/30...
🔄 Processing product 6/30...
🔄 Processing product 7/30...
🔄 Processing product 8/30...
🔄 Processing product 9/30...
🔄 Processing product 10/30...
🔄 Processing product 11/30...
🔄 Processing product 12/30...
🔄 Processing product 13/30...
🔄 Processing product 14/30...
🔄 Processing product 15/30...
🔄 Processing product 16/30...
🔄 Processing product 17/30...
🔄 Processing product 18/30...
🔄 Processing product 19/30...
🔄 Processing product 20/30...
🔄 Processing product 21/30...
🔄 Processing product 22/30...
🔄 Processing product 23/30...
🔄 Processing product 24/30...
🔄 Processing product 25/30...
🔄 Processing product 26/3

## 4. Map dữ liệu và Generate SQL INSERT

In [33]:
# Thực thi file product.insert.sql
try:
    print("⚙️ Đang kết nối database...")
    
    if not conn:
        conn = psycopg2.connect(SUPABASE_CONNECTION_STRING)
        cursor = conn.cursor()
        print("✅ Kết nối thành công!\n")
    
    # Đọc file SQL
    with open('product_addition.insert.sql', 'r', encoding='utf-8') as f:
        sql_content = f.read()
    
    # Tách các câu lệnh INSERT (bỏ qua comments)
    sql_statements = []
    for statement in sql_content.split('\n\n'):
        statement = statement.strip()
        if statement and not statement.startswith('--'):
            sql_statements.append(statement)
    
    print(f"📋 Tìm thấy {len(sql_statements)} câu lệnh INSERT\n")
    print("🚀 Bắt đầu thực thi...\n")
    
    success_count = 0
    error_count = 0
    
    for idx, statement in enumerate(sql_statements, 1):
        try:
            cursor.execute(statement)
            conn.commit()
            success_count += 1
            print(f"✅ [{idx}/{len(sql_statements)}] Inserted successfully")
        except Exception as e:
            conn.rollback()
            error_count += 1
            print(f"❌ [{idx}/{len(sql_statements)}] Error: {str(e)[:100]}")
    
    print(f"\n📊 Kết quả:")
    print(f"   ✅ Thành công: {success_count}")
    print(f"   ❌ Thất bại: {error_count}")
    print(f"   📊 Tổng: {len(sql_statements)}")
    
    cursor.close()
    conn.close()
    print("\n🔒 Đã đóng kết nối database")
    
except FileNotFoundError:
    print("❌ Không tìm thấy file product.insert.sql")
except Exception as e:
    print(f"❌ Lỗi: {e}")
    import traceback
    traceback.print_exc()

⚙️ Đang kết nối database...
📋 Tìm thấy 30 câu lệnh INSERT

🚀 Bắt đầu thực thi...

✅ [1/30] Inserted successfully
✅ [2/30] Inserted successfully
✅ [3/30] Inserted successfully
✅ [4/30] Inserted successfully
✅ [5/30] Inserted successfully
✅ [6/30] Inserted successfully
✅ [7/30] Inserted successfully
✅ [8/30] Inserted successfully
✅ [9/30] Inserted successfully
✅ [10/30] Inserted successfully
✅ [11/30] Inserted successfully
✅ [12/30] Inserted successfully
✅ [13/30] Inserted successfully
✅ [14/30] Inserted successfully
✅ [15/30] Inserted successfully
✅ [16/30] Inserted successfully
✅ [17/30] Inserted successfully
✅ [18/30] Inserted successfully
✅ [19/30] Inserted successfully
✅ [20/30] Inserted successfully
✅ [21/30] Inserted successfully
✅ [22/30] Inserted successfully
✅ [23/30] Inserted successfully
✅ [24/30] Inserted successfully
✅ [25/30] Inserted successfully
✅ [26/30] Inserted successfully
✅ [27/30] Inserted successfully
✅ [28/30] Inserted successfully
✅ [29/30] Inserted successfully

## 5. Execute SQL File